In [1]:
# Skriptas originalių Zive įrašų pertvarkymui į labiau tinkamą formą klasifikatorių kūrimui.
# 
# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
# Šitame variante keičiamas list.json ir df_transl failai
#
# iš originalių Zive įrašų sukuriami įrašai *.npy,
# Zive ekg įrašo identikatoriai userId, file_name pakeičiami į virtualią numeraciją:
# userId pakeičiamas į userNr - skaičiumi, pradedant nuo 1000, įrašo įdentifikatorius file_name pakeičiamas
# į įrašo eilės numerį registrationNr, lygų 0,1,... ir suformuojamas virtualus įrašo failo vardas SubjCode.
# Konversijai ir saugojimui panaudojamas df masyvas df_transl.
# Koreguojamas list.json failas, papildant atributu SubjCode. Anotacijų failai *.json - nekeičiami.


import shutil, sys
from icecream import ic
from pathlib import Path
import pandas as pd
import numpy as np

import json
from zive_util_vu import zive_read_file_1ch, create_dir, create_SubjCode, get_recNr


import warnings
warnings.filterwarnings("ignore")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

# Vietinės talpyklos aplankas
db_folder = 'DUOM_2022_RUDUO_2'

# Nuoroda į aplanką su EKG duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

rec_dir = Path(db_path,'records_selected')
rec_list = 'list.json'

# Nuoroda į aplanką su transformuotu EKG duomenų rinkiniu
rec_dir_npy = Path(db_path, 'records_npy')
rec_list_npy = 'list_npy.json'

# Paliekamų anotacijų sąrašas
annot_list = ['N','S','V','U']

# Diskretizavimo dažnis
fs_zive = 200

print("\nSkriptas zive įrašų transformacijai į npy formatą ir")
print("Zive ekg įrašo identikatorių userId, file_name pakeitimui į virtualią numeraciją\n")

print("Išeities duomenys skaitomi iš:", rec_dir)
print("EKG įrašų atributų sąrašas:", rec_list)

print("Transformuoti duomenys rašomi į:", rec_dir_npy)
print("Atnaujintas EKG įrašų atributų sąrašas:", rec_list_npy)
print("\n")


OS in my system :  win32

Skriptas zive įrašų transformacijai į npy formatą ir
Zive ekg įrašo identikatorių userId, file_name pakeitimui į virtualią numeraciją

Išeities duomenys skaitomi iš: D:\DI\DUOM_2022_RUDUO_2\records_selected
EKG įrašų atributų sąrašas: list.json
Transformuoti duomenys rašomi į: D:\DI\DUOM_2022_RUDUO_2\records_npy
Atnaujintas EKG įrašų atributų sąrašas: list_npy.json




In [2]:

# Sukūriame aplanką EKG sekų įrašymui
create_dir(rec_dir_npy)

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 20)
pd.set_option('display.width', 2000)

# Nuskaitome Zive įrašų talpykloje laikomų įrašų sąrašą
file_path = Path(rec_dir, rec_list)
with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
    data = json.loads(f.read())

df_list = pd.json_normalize(data, record_path =['data'])

# print(df_list)
# Sugrupuojame įrašus į to paties paciento grupes
# grouped = df_list.groupby(['userId','file_name'])
# print(grouped)
# print("\n")

SubjCodes = []

# Perrašome įrašus nauju formatu į kitą folderį
# for key in grouped.groups:
for ind, row in df_list.iterrows():
    # userId = key[0]
    # file_name = str(key[1])
    userId = row['userId']
    recordingId = row['recordingId']
    file_name = row['file_name']
    # print(userId, file_name)

    # Sukūriame userNr, recordingNr, Subjcode, papildome žodyną transl.csv  
    userNr, recordingNr = get_recNr(rec_dir_npy, userId, recordingId, file_name)
    # print(userNr, recordingNr)
    SubjCode = create_SubjCode(userNr, recordingNr)  
    # print("SubjCode: ", SubjCode, userNr, file_name)

    SubjCodes.append(SubjCode)

    file_path = Path(rec_dir, file_name)
    signal = zive_read_file_1ch(file_path)  

    file_path = Path(rec_dir_npy, str(SubjCode) + '.npy')
    with open(file_path, 'wb') as f:
        np.save(f, signal)

    src = Path(rec_dir, file_name + '.json')
    dst = Path(rec_dir_npy, str(SubjCode) + '.json')
    shutil.copy2(src, dst)

# print(SubjCodes)

# Papildome įrašų sarašą df_list stulpeliu SubjCode
df_list.insert(0, 'SubjCode', SubjCodes, True)

print(df_list)

# Įrašome failų sąrašą į diską
file_path = Path(rec_dir_npy, rec_list_npy)       
df_list.to_json(file_path, orient = 'table', index=False)
print(f'\nEKG įrašų atributų sąrašas įrašytas:  {file_path}')


info = {
    'db_folder':db_folder,
    'fs': fs_zive,
    'SubjCodes':SubjCodes,
    'annot_list':annot_list
    }

file_name = Path(rec_dir_npy,'info_create_z.json')
with open(file_name, 'w') as f:
    json.dump(info, f)
    
print("\nSchemos parametrai įrašyti į failą: ", file_name, "\n" )

all_subjcode_lst = info['SubjCodes'] # pacientų įrašų sąrašas
file_path = Path(rec_dir_npy,'all_subjcode_lst_z.csv')
np.savetxt(file_path, np.array(all_subjcode_lst), delimiter=',', fmt='%d')
print("\nPacientų įrašų kodai įrašyti į failą: ", file_path )


Directory 'D:\DI\DUOM_2022_RUDUO_2\records_npy' created successfully
      SubjCode    file_name                    userId               recordingId     N  S  V  U  incl  flag               recorded_at
0        10000  1630673.825  6034c808d6c2740008035ede  6133101f3d08d40385cdc8ce   762  4  0  0     0     0  2021-09-03T12:57:05.000Z
1        10001  1626941.468  6034c808d6c2740008035ede  60f945e187cf66e97e37810e   859  2  0  0     0     0  2021-07-22T08:11:08.000Z
2        10002  1632923.661  6034c808d6c2740008035ede  615516b62312e50c3f168e38  4849  1  0  0     0     0  2021-09-29T13:54:21.000Z
3        10010  1621694.321  60a917b354352a3df86dc1f2  60a92aa454352a03116dc1f5  6009  0  9  0     0     0  2021-05-22T14:38:41.000Z
4        10011  1626330.744  60a917b354352a3df86dc1f2  60efffba87cf660aac377fe7   788  5  0  4     0     0  2021-07-15T06:32:24.000Z
...        ...          ...                       ...                       ...   ... .. .. ..   ...   ...                       ...


In [3]:
# https://www.geeksforgeeks.org/python-shutil-copy2-method/#:~:text=shutil.copy
# 2%20%28%29%20method%20in%20Python%20is%20used%20to,destination%20can%20be%20a%20file%20or%20a%20directory.
from zive_util_vu import split_SubjCode

# print(df_list)


SubjCodes = list(df_list['SubjCode'])
userNrs = [split_SubjCode(SubjCode)[0] for SubjCode in SubjCodes]
df_list['userNr'] = userNrs
# print(userNrs)

# Susirandame anotacijų pasiskirstymą per pacientus ir pacientų skaičių
df_sum = df_list.groupby(['userNr'],sort = False).sum()
# print(df_sum)
# https://sparkbyexamples.com/pandas/pandas-groupby-sum-examples/
count = df_list['userNr'].value_counts()
print(f'\nViso pacientų: {len(count)}  EKG įrašų: {len(df_list)}')
print(f'\nĮrašų pasiskirstymas per pacientus')
count = count.rename("count")
frames = [df_sum, count]
result = pd.concat(frames, axis = 1)
result.index.rename ('userNr', inplace= True)
result = result.reset_index()

result.drop(labels=['incl'], axis=1, inplace=True)
result.drop(labels=['flag'], axis=1, inplace=True)
result.drop(labels=['SubjCode'], axis=1, inplace=True)
# https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/

print(result)

suma = result.iloc[:,1:].sum()
print('\n',' '*5, 'sum: ',str(suma.tolist())[1:-1])
total = suma.sum()   # pataisyti, kad sumuotų tik pūpsnius
print(' '*5, 'total: ', total)



Viso pacientų: 102  EKG įrašų: 1087

Įrašų pasiskirstymas per pacientus
     userNr      N     S     V  U  count
0      1000   6470     7     0  0      3
1      1001   9970    16    10  5      6
2      1002   7238    11    72  0      7
3      1003   1186     0     1  0      2
4      1004   1858     3     0  0      3
5      1005  10140    30     2  0     18
6      1006   2498     1     2  0      4
7      1007   1260     1     0  0      2
8      1008  18638    32   233  5     25
9      1009  21721   126     0  5     29
10     1010   1518     1     0  0      2
11     1011   2105     1     4  0      3
12     1012   1885     5     4  0      3
13     1013  28108     2  3760  0     41
14     1014    816     0     0  0      1
15     1015  38541     0  1156  0     65
16     1016   1536     1     0  0      2
17     1017    518     0     0  0      1
18     1018   5738     0     9  0      9
19     1019  87673     4  4460  2    136
20     1020   1928     0     2  0      3
21     1021    663     0 